<h1>CycleGAN for Style Transfer</h1>
<h3> Imports </h3>

In [1]:
#Python imports
import torch
import torch.nn as tnn
from torch.autograd import Variable
import sys
import itertools
from torch.utils.data import DataLoader
from Dataset import ImageSet
import datetime
import torchvision.utils as tv
import os
import matplotlib.pyplot as plt
import numpy as np

#Import network elements
import network_elements as net

#stdout code blocks reload's print statement
#prevstd = sys.stdout
#sys.stdout = None
reload(net)
print("Reloading net")
#sys.stdout = prevstd

Reloading net


<h3>Set parameters</h3>

In [2]:
#The number of channels in the input image
im_in_channels = 3

#The number of channels in the output image
im_out_channels = 3

#The size of the largest size of the input image
im_size = 256

LEARNING_RATE = 0.0002

LR_DECAY_START = 100
LR_DECAY_END = 200
NUM_EPOCHS = 200

load_partial_net = True
CURR_EPOCH = 0

<h3>Import dataset</h3>

In [3]:
imageSet = ImageSet();
dataset = "horse2zebra"
imageSet.loadData(dataset, 'train', im_size)
imgLoader = DataLoader(imageSet,shuffle=True)

./datasets/horse2zebra
Finished loading data


<h3>Define the CycleGAN network</h3>

In [4]:
#G : X->Y
G = net.Mapping(im_in_channels, im_out_channels, im_size)

#F : Y->X
F = net.Mapping(im_out_channels, im_out_channels, im_size)

#G mapping discriminator
Dx = net.Discriminator(im_in_channels)

#F mapping discriminator
Dy = net.Discriminator(im_out_channels)

#Initialize CUDA implementations
G.cuda();
F.cuda();
Dx.cuda();
Dy.cuda();

#Define losses
cycle_loss = tnn.L1Loss(); #Cycle-consistency loss
gan_loss = tnn.MSELoss(); #Adversarial loss
identity_loss = tnn.L1Loss(); #Prevents image tinting

<h3>Initialize weights for each network</h3>

In [5]:
# norm_weights function applies Gaussian norm weights with mean 0 and stddev 0.02
G.apply(net.conv_norm_weights);
F.apply(net.conv_norm_weights);
Dx.apply(net.conv_norm_weights);
Dy.apply(net.conv_norm_weights);

<h3>Set optimizer parameters</h3>

In [6]:
#simplified version has no pooling, paper does not, paper code does...
# REMOVE

gan_opt = torch.optim.Adam(itertools.chain(G.parameters(), F.parameters()),
                           lr = LEARNING_RATE)
dx_opt = torch.optim.Adam(Dx.parameters(), lr = LEARNING_RATE)
dy_opt = torch.optim.Adam(Dy.parameters(), lr = LEARNING_RATE)

The paper uses Adam optimizer with a learning rate of 0.0002 for the first 100 epochs that linearly decays to 0 over the next 100 epochs.

<h3> Initialize training variables </h3>

In [7]:
x = torch.cuda.FloatTensor(1, im_in_channels, im_size, im_size)
y = torch.cuda.FloatTensor(1, im_out_channels, im_size, im_size)

# Variable wrapper is depricated in >0.4
dReal = Variable(torch.cuda.FloatTensor(1).fill_(1.0), requires_grad = False)
dFake = Variable(torch.cuda.FloatTensor(1).fill_(0.0), requires_grad = False)

buffX_Y = net.MapBuffer()
buffY_X = net.MapBuffer()

if not os.path.isdir("./img/"):
    os.mkdir("./img/")
    
if not os.path.isdir("./log/"):
    os.mkdir("./log/")
    
if os.path.isfile("./log/loss.log") and not load_partial_net:    
    os.remove("./log/loss.log")
    
logfile = open("./log/loss.log","a")

<h3> Load Partial Net </h3>

In [8]:
if (load_partial_net):
    if os.path.isfile("./model/model_info.txt"):
        model_file = open("./model/model_info.txt", "r")
        CURR_EPOCH = int(model_file.read())
        model_file.close()
    G.load_state_dict(torch.load('./model/G.data'))
    F.load_state_dict(torch.load('./model/F.data'))
    Dx.load_state_dict(torch.load('./model/Dx.data'))
    Dy.load_state_dict(torch.load('./model/Dy.data'))
    gan_opt.load_state_dict(torch.load('./model/gan_opt.data'))
    dx_opt.load_state_dict(torch.load('./model/dx_opt.data'))
    dy_opt.load_state_dict(torch.load('./model/dy_opt.data'))
    
last_epoch = -1
if (load_partial_net):
    last_epoch = CURR_EPOCH-1

gan_lr = torch.optim.lr_scheduler.LambdaLR(gan_opt, net.LRDecay(LR_DECAY_START, LR_DECAY_END).step, last_epoch)
dx_lr = torch.optim.lr_scheduler.LambdaLR(dx_opt, net.LRDecay(LR_DECAY_START, LR_DECAY_END).step, last_epoch)
dy_lr = torch.optim.lr_scheduler.LambdaLR(dy_opt, net.LRDecay(LR_DECAY_START, LR_DECAY_END).step, last_epoch)

<h3> Training </h3>

In [ ]:
start_time = datetime.datetime.now()
for i in range(CURR_EPOCH,NUM_EPOCHS):
    for j, img in enumerate(imgLoader):
        elapsed_time = datetime.datetime.now()-start_time
        print("Epoch %d/%d, %d/%d, %s"%(i,LR_DECAY_END,j,len(imageSet),str(elapsed_time)))
        src_x = Variable(x.copy_(img['x']))
        src_y = Variable(y.copy_(img['y']))
        
        gan_opt.zero_grad()
        
        #identity Loss
        mapY_Y = G(src_y)
        lossY_Y = identity_loss(mapY_Y, src_y)*5.0
        
        mapX_X = F(src_x)
        lossX_X = identity_loss(mapX_X, src_x)*5.0
        
        #GAN Loss
        mapX_Y = G(src_x)
        dX_Y = Dx(mapX_Y)
        lossX_Y = gan_loss(dX_Y, dReal)
        
        mapY_X = F(src_y)
        dY_X = Dy(mapY_X)
        lossY_X = gan_loss(dY_X, dReal)
        
        #Cycle Consistency Loss
        mapX_Y_X = F(mapX_Y)
        lossX_Y_X = cycle_loss(mapX_Y_X, src_x)*10.0
        
        mapY_X_Y = G(mapY_X)
        lossY_X_Y = cycle_loss(mapY_X_Y, src_y)*10.0
        
        mapLoss = lossY_Y + lossX_X + lossX_Y + lossY_X + lossX_Y_X + lossY_X_Y
        mapLoss.backward()
        
        gan_opt.step()
        
        #Dx
        dx_opt.zero_grad()
        
        dX_res = Dx(src_x)
        lossX = gan_loss(dX_res, dReal)
        
        mapY_X_old = buffY_X.cycle(mapY_X)
        dY_X = Dx(mapY_X_old.detach())
        lossY_X = gan_loss(dY_X, dFake)
        
        dxLoss = (lossX + lossY_X)*0.5
        dxLoss.backward()
        
        dx_opt.step()
        
        #Dy
        dy_opt.zero_grad()
        
        dY_res = Dy(src_y)
        lossY = gan_loss(dY_res, dReal)
        
        mapX_Y_old = buffX_Y.cycle(mapX_Y)
        dX_Y = Dy(mapX_Y_old.detach())
        lossX_Y = gan_loss(dX_Y, dFake)
        
        dyLoss = (lossY + lossX_Y)*0.5
        dyLoss.backward()
        
        dy_opt.step()     
        
        
        
        logfile.write("%d,%d,%f,%f,%f,%f,%f\n"%(i,j,mapLoss.data[0], 
                                          (lossX_X+lossY_Y).data[0], 
                                          (lossX_Y+lossY_X).data[0], 
                                          (lossX_Y_X+lossY_X_Y).data[0], 
                                          (dxLoss+dyLoss).data[0]))
        
    tv.save_image(src_x.data, './img/src_x_%d.jpg'%(i))
    tv.save_image(src_y.data, './img/src_y_%d.jpg'%(i))
    tv.save_image(mapX_Y.data, './img/mapX_Y_%d.jpg'%(i))
    tv.save_image(mapY_X.data, './img/mapY_X_%d.jpg'%(i))
    gan_lr.step()
    dx_lr.step()
    dy_lr.step()
    

    if not os.path.isdir("./model/"):
        os.mkdir("./model/")
    model_file = open("./model/model_info.txt", "w")
    model_file.write("%d"%(i+1))
    model_file.close()
    torch.save(G.state_dict(), './model/G.data')
    torch.save(F.state_dict(), './model/F.data')
    torch.save(Dx.state_dict(), './model/Dx.data')
    torch.save(Dy.state_dict(), './model/Dy.data')
    torch.save(gan_opt.state_dict(), './model/gan_opt.data')
    torch.save(dx_opt.state_dict(), './model/dx_opt.data')
    torch.save(dy_opt.state_dict(), './model/dy_opt.data')

Epoch 65/200, 0/1334, 0:00:00.030945
Epoch 65/200, 1/1334, 0:00:03.018181
Epoch 65/200, 2/1334, 0:00:03.634746
Epoch 65/200, 3/1334, 0:00:04.201826
Epoch 65/200, 4/1334, 0:00:04.797777
Epoch 65/200, 5/1334, 0:00:05.406745
Epoch 65/200, 6/1334, 0:00:05.985164
Epoch 65/200, 7/1334, 0:00:06.588643
Epoch 65/200, 8/1334, 0:00:07.195489
Epoch 65/200, 9/1334, 0:00:07.803565
Epoch 65/200, 10/1334, 0:00:08.392762
Epoch 65/200, 11/1334, 0:00:09.008831
Epoch 65/200, 12/1334, 0:00:09.602510
Epoch 65/200, 13/1334, 0:00:10.177329
Epoch 65/200, 14/1334, 0:00:10.797147
Epoch 65/200, 15/1334, 0:00:11.411056
Epoch 65/200, 16/1334, 0:00:11.994854
Epoch 65/200, 17/1334, 0:00:12.590645
Epoch 65/200, 18/1334, 0:00:13.195337
Epoch 65/200, 19/1334, 0:00:13.790998
Epoch 65/200, 20/1334, 0:00:14.408134
Epoch 65/200, 21/1334, 0:00:15.015304
Epoch 65/200, 22/1334, 0:00:15.590780
Epoch 65/200, 23/1334, 0:00:16.200048
Epoch 65/200, 24/1334, 0:00:16.786841
Epoch 65/200, 25/1334, 0:00:17.412291
Epoch 65/200, 26/1334,

Epoch 65/200, 213/1334, 0:02:10.191272
Epoch 65/200, 214/1334, 0:02:10.795413
Epoch 65/200, 215/1334, 0:02:11.379985
Epoch 65/200, 216/1334, 0:02:12.022189
Epoch 65/200, 217/1334, 0:02:12.607498
Epoch 65/200, 218/1334, 0:02:13.203804
Epoch 65/200, 219/1334, 0:02:13.802088
Epoch 65/200, 220/1334, 0:02:14.402430
Epoch 65/200, 221/1334, 0:02:15.003973
Epoch 65/200, 222/1334, 0:02:15.596245
Epoch 65/200, 223/1334, 0:02:16.182832
Epoch 65/200, 224/1334, 0:02:16.813510
Epoch 65/200, 225/1334, 0:02:17.422191
Epoch 65/200, 226/1334, 0:02:18.005976
Epoch 65/200, 227/1334, 0:02:18.596528
Epoch 65/200, 228/1334, 0:02:19.201988
Epoch 65/200, 229/1334, 0:02:19.787750
Epoch 65/200, 230/1334, 0:02:20.403272
Epoch 65/200, 231/1334, 0:02:21.014492
Epoch 65/200, 232/1334, 0:02:21.594493
Epoch 65/200, 233/1334, 0:02:22.192913
Epoch 65/200, 234/1334, 0:02:22.801598
Epoch 65/200, 235/1334, 0:02:23.409530
Epoch 65/200, 236/1334, 0:02:24.033830
Epoch 65/200, 237/1334, 0:02:24.618983
Epoch 65/200, 238/1334, 0

Epoch 65/200, 424/1334, 0:04:16.693251
Epoch 65/200, 425/1334, 0:04:17.301567
Epoch 65/200, 426/1334, 0:04:17.889399
Epoch 65/200, 427/1334, 0:04:18.503575
Epoch 65/200, 428/1334, 0:04:19.091086
Epoch 65/200, 429/1334, 0:04:19.694276
Epoch 65/200, 430/1334, 0:04:20.294780
Epoch 65/200, 431/1334, 0:04:20.886309
Epoch 65/200, 432/1334, 0:04:21.492973
Epoch 65/200, 433/1334, 0:04:22.087344
Epoch 65/200, 434/1334, 0:04:22.685467
Epoch 65/200, 435/1334, 0:04:23.295342
Epoch 65/200, 436/1334, 0:04:23.879095
Epoch 65/200, 437/1334, 0:04:24.493548
Epoch 65/200, 438/1334, 0:04:25.099121
Epoch 65/200, 439/1334, 0:04:25.725812
Epoch 65/200, 440/1334, 0:04:26.302409
Epoch 65/200, 441/1334, 0:04:26.881611
Epoch 65/200, 442/1334, 0:04:27.475966
Epoch 65/200, 443/1334, 0:04:28.101007
Epoch 65/200, 444/1334, 0:04:28.713848
Epoch 65/200, 445/1334, 0:04:29.312876
Epoch 65/200, 446/1334, 0:04:29.917540
Epoch 65/200, 447/1334, 0:04:30.534202
Epoch 65/200, 448/1334, 0:04:31.111270
Epoch 65/200, 449/1334, 0

Epoch 65/200, 635/1334, 0:06:23.312057
Epoch 65/200, 636/1334, 0:06:23.918524
Epoch 65/200, 637/1334, 0:06:24.514086
Epoch 65/200, 638/1334, 0:06:25.106150
Epoch 65/200, 639/1334, 0:06:25.708710
Epoch 65/200, 640/1334, 0:06:26.333563
Epoch 65/200, 641/1334, 0:06:26.928454
Epoch 65/200, 642/1334, 0:06:27.515109
Epoch 65/200, 643/1334, 0:06:28.108417
Epoch 65/200, 644/1334, 0:06:28.677690
Epoch 65/200, 645/1334, 0:06:29.299112
Epoch 65/200, 646/1334, 0:06:29.895465
Epoch 65/200, 647/1334, 0:06:30.482758
Epoch 65/200, 648/1334, 0:06:31.095981
Epoch 65/200, 649/1334, 0:06:31.688206
Epoch 65/200, 650/1334, 0:06:32.307771
Epoch 65/200, 651/1334, 0:06:32.898891
Epoch 65/200, 652/1334, 0:06:33.505348
Epoch 65/200, 653/1334, 0:06:34.090993
Epoch 65/200, 654/1334, 0:06:34.678230
Epoch 65/200, 655/1334, 0:06:35.302497
Epoch 65/200, 656/1334, 0:06:35.908605
Epoch 65/200, 657/1334, 0:06:36.490160
Epoch 65/200, 658/1334, 0:06:37.087725
Epoch 65/200, 659/1334, 0:06:37.684143
Epoch 65/200, 660/1334, 0

Epoch 65/200, 846/1334, 0:08:29.781756
Epoch 65/200, 847/1334, 0:08:30.394131
Epoch 65/200, 848/1334, 0:08:30.988285
Epoch 65/200, 849/1334, 0:08:31.601725
Epoch 65/200, 850/1334, 0:08:32.185800
Epoch 65/200, 851/1334, 0:08:32.812744
Epoch 65/200, 852/1334, 0:08:33.385293
Epoch 65/200, 853/1334, 0:08:34.016466
Epoch 65/200, 854/1334, 0:08:34.590967
Epoch 65/200, 855/1334, 0:08:35.193806
Epoch 65/200, 856/1334, 0:08:35.811800
Epoch 65/200, 857/1334, 0:08:36.415217
Epoch 65/200, 858/1334, 0:08:37.004369
Epoch 65/200, 859/1334, 0:08:37.583177
Epoch 65/200, 860/1334, 0:08:38.185742
Epoch 65/200, 861/1334, 0:08:38.789379
Epoch 65/200, 862/1334, 0:08:39.392083
Epoch 65/200, 863/1334, 0:08:39.979334
Epoch 65/200, 864/1334, 0:08:40.597041
Epoch 65/200, 865/1334, 0:08:41.193353
Epoch 65/200, 866/1334, 0:08:41.798848
Epoch 65/200, 867/1334, 0:08:42.412713
Epoch 65/200, 868/1334, 0:08:43.001009
Epoch 65/200, 869/1334, 0:08:43.614296
Epoch 65/200, 870/1334, 0:08:44.190790
Epoch 65/200, 871/1334, 0

...

Epoch 79/200, 0/1334, 3:06:49.331088
Epoch 79/200, 1/1334, 3:06:49.910725
Epoch 79/200, 2/1334, 3:06:50.481840
Epoch 79/200, 3/1334, 3:06:51.096170
Epoch 79/200, 4/1334, 3:06:51.679358
Epoch 79/200, 5/1334, 3:06:52.292178
Epoch 79/200, 6/1334, 3:06:52.896756
Epoch 79/200, 7/1334, 3:06:53.476580
Epoch 79/200, 8/1334, 3:06:54.093910
Epoch 79/200, 9/1334, 3:06:54.681660
Epoch 79/200, 10/1334, 3:06:55.298256
Epoch 79/200, 11/1334, 3:06:55.887354
Epoch 79/200, 12/1334, 3:06:56.496225
Epoch 79/200, 13/1334, 3:06:57.086234
Epoch 79/200, 14/1334, 3:06:57.680102
Epoch 79/200, 15/1334, 3:06:58.291311
Epoch 79/200, 16/1334, 3:06:58.888281
Epoch 79/200, 17/1334, 3:06:59.491825
Epoch 79/200, 18/1334, 3:07:00.096796
Epoch 79/200, 19/1334, 3:07:00.682260
Epoch 79/200, 20/1334, 3:07:01.300721
Epoch 79/200, 21/1334, 3:07:01.889203
Epoch 79/200, 22/1334, 3:07:02.489658
Epoch 79/200, 23/1334, 3:07:03.078845
Epoch 79/200, 24/1334, 3:07:03.680597
Epoch 79/200, 25/1334, 3:07:04.297909
Epoch 79/200, 26/1334,

Epoch 79/200, 213/1334, 3:08:57.107045
Epoch 79/200, 214/1334, 3:08:57.702657
Epoch 79/200, 215/1334, 3:08:58.314009
Epoch 79/200, 216/1334, 3:08:58.895249
Epoch 79/200, 217/1334, 3:08:59.502709
Epoch 79/200, 218/1334, 3:09:00.108191
Epoch 79/200, 219/1334, 3:09:00.705411
Epoch 79/200, 220/1334, 3:09:01.301046
Epoch 79/200, 221/1334, 3:09:01.911335
Epoch 79/200, 222/1334, 3:09:02.505500
Epoch 79/200, 223/1334, 3:09:03.099888
Epoch 79/200, 224/1334, 3:09:03.710916
Epoch 79/200, 225/1334, 3:09:04.300743
Epoch 79/200, 226/1334, 3:09:04.898633
Epoch 79/200, 227/1334, 3:09:05.510936
Epoch 79/200, 228/1334, 3:09:06.114162
Epoch 79/200, 229/1334, 3:09:06.716460
Epoch 79/200, 230/1334, 3:09:07.304299
Epoch 79/200, 231/1334, 3:09:07.902761
Epoch 79/200, 232/1334, 3:09:08.506697
Epoch 79/200, 233/1334, 3:09:09.093598
Epoch 79/200, 234/1334, 3:09:09.703757
Epoch 79/200, 235/1334, 3:09:10.302689
Epoch 79/200, 236/1334, 3:09:10.915253
Epoch 79/200, 237/1334, 3:09:11.511539
Epoch 79/200, 238/1334, 3

Epoch 79/200, 424/1334, 3:11:03.295216
Epoch 79/200, 425/1334, 3:11:03.903023
Epoch 79/200, 426/1334, 3:11:04.499197
Epoch 79/200, 427/1334, 3:11:05.100554
Epoch 79/200, 428/1334, 3:11:05.715405
Epoch 79/200, 429/1334, 3:11:06.307219
Epoch 79/200, 430/1334, 3:11:06.906375
Epoch 79/200, 431/1334, 3:11:07.505980
Epoch 79/200, 432/1334, 3:11:08.083448
Epoch 79/200, 433/1334, 3:11:08.702850
Epoch 79/200, 434/1334, 3:11:09.307448
Epoch 79/200, 435/1334, 3:11:09.884555
Epoch 79/200, 436/1334, 3:11:10.507519
Epoch 79/200, 437/1334, 3:11:11.090500
Epoch 79/200, 438/1334, 3:11:11.678246
Epoch 79/200, 439/1334, 3:11:12.281923
Epoch 79/200, 440/1334, 3:11:12.877032
Epoch 79/200, 441/1334, 3:11:13.483326
Epoch 79/200, 442/1334, 3:11:14.078220
Epoch 79/200, 443/1334, 3:11:14.682964
Epoch 79/200, 444/1334, 3:11:15.275131
Epoch 79/200, 445/1334, 3:11:15.866255
Epoch 79/200, 446/1334, 3:11:16.473836
Epoch 79/200, 447/1334, 3:11:17.076513
Epoch 79/200, 448/1334, 3:11:17.679744
Epoch 79/200, 449/1334, 3

Epoch 79/200, 635/1334, 3:13:09.888873
Epoch 79/200, 636/1334, 3:13:10.480501
Epoch 79/200, 637/1334, 3:13:11.088707
Epoch 79/200, 638/1334, 3:13:11.680228
Epoch 79/200, 639/1334, 3:13:12.286485
Epoch 79/200, 640/1334, 3:13:12.880345
Epoch 79/200, 641/1334, 3:13:13.486582
Epoch 79/200, 642/1334, 3:13:14.095914
Epoch 79/200, 643/1334, 3:13:14.692948
Epoch 79/200, 644/1334, 3:13:15.296928
Epoch 79/200, 645/1334, 3:13:15.896476
Epoch 79/200, 646/1334, 3:13:16.489569
Epoch 79/200, 647/1334, 3:13:17.095768
Epoch 79/200, 648/1334, 3:13:17.672866
Epoch 79/200, 649/1334, 3:13:18.310742
Epoch 79/200, 650/1334, 3:13:18.933611
Epoch 79/200, 651/1334, 3:13:19.518121
Epoch 79/200, 652/1334, 3:13:20.092491
Epoch 79/200, 653/1334, 3:13:20.717681
Epoch 79/200, 654/1334, 3:13:21.313820
Epoch 79/200, 655/1334, 3:13:21.913232
Epoch 79/200, 656/1334, 3:13:22.533967
Epoch 79/200, 657/1334, 3:13:23.130575
Epoch 79/200, 658/1334, 3:13:23.725909
Epoch 79/200, 659/1334, 3:13:24.328448
Epoch 79/200, 660/1334, 3

Epoch 79/200, 846/1334, 3:15:16.309477
Epoch 79/200, 847/1334, 3:15:16.902435
Epoch 79/200, 848/1334, 3:15:17.508988
Epoch 79/200, 849/1334, 3:15:18.110119
Epoch 79/200, 850/1334, 3:15:18.709998
Epoch 79/200, 851/1334, 3:15:19.297058
Epoch 79/200, 852/1334, 3:15:19.908053
Epoch 79/200, 853/1334, 3:15:20.491928
Epoch 79/200, 854/1334, 3:15:21.094518
Epoch 79/200, 855/1334, 3:15:21.679986
Epoch 79/200, 856/1334, 3:15:22.274843
Epoch 79/200, 857/1334, 3:15:22.887938
Epoch 79/200, 858/1334, 3:15:23.503122
Epoch 79/200, 859/1334, 3:15:24.104255
Epoch 79/200, 860/1334, 3:15:24.711985
Epoch 79/200, 861/1334, 3:15:25.305644
Epoch 79/200, 862/1334, 3:15:25.903464
Epoch 79/200, 863/1334, 3:15:26.488935
Epoch 79/200, 864/1334, 3:15:27.119676
Epoch 79/200, 865/1334, 3:15:27.703050
Epoch 79/200, 866/1334, 3:15:28.283189
Epoch 79/200, 867/1334, 3:15:28.918767
Epoch 79/200, 868/1334, 3:15:29.504414
Epoch 79/200, 869/1334, 3:15:30.110596
Epoch 79/200, 870/1334, 3:15:30.705483
Epoch 79/200, 871/1334, 3

In [ ]:
close(logfile)